<a href="https://colab.research.google.com/github/esephaneli/BIST_AI_Terminal_Demo/blob/main/BIST_AI_Terminal_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 BIST AI Terminal
> **Amazon Chronos + Google Gemini + Streamlit**

Bu notebook, Amazon'un açık kaynak zaman serisi modeli **Chronos**'u kullanarak BIST hisseleri ve kripto paralar için:
- 🔮 **Probabilistic Forecast** (5 günlük tahmin + güven bandı)
- 🚨 **Anomaly Detection** (beklenen bandın dışına çıkan fiyat hareketleri)
- 🤖 **AI Analiz** (Gemini ile otomatik Türkçe yorum)
- 📈 **Teknik Analiz** (RSI, MACD, Bollinger Bands)

yapan bir terminal geliştirme sürecini göstermektedir.

---
**Stack:** `chronos-forecasting` • `yfinance` • `pandas-ta` • `plotly` • `google-genai` • `streamlit`

⚠️ *Bu araç yatırım tavsiyesi değildir. Eğitim amaçlıdır.*

## 🏗️ Mimari

```
┌─────────────────────────────────────────────────────┐
│                   DATA LAYER                        │
│         yfinance → OHLCV + Teknik İndikatörler      │
└─────────────────────┬───────────────────────────────┘
                      ↓
┌─────────────────────────────────────────────────────┐
│               ANALYSIS LAYER                        │
│  Amazon Chronos  →  Probabilistic Forecast          │
│                  →  Anomaly Score                   │
│  Google Gemini   →  Türkçe Narrative                │
└─────────────────────┬───────────────────────────────┘
                      ↓
┌─────────────────────────────────────────────────────┐
│                    UI LAYER                         │
│      Streamlit Dashboard (Bloomberg-style)          │
└─────────────────────────────────────────────────────┘
```

### Neden Chronos?
Chronos, Amazon tarafından geliştirilen ve **zero-shot** çalışabilen bir time series foundation model'idir.
Fine-tune gerektirmeden herhangi bir zaman serisi üzerinde probabilistic forecast üretebilir.
Bu projede fiyat serisi yerine **anomaly detection** için kullandık — modelin asıl gücü burada.

## 1️⃣ Kurulum

In [ ]:
# Gerekli paketleri kur
!pip install chronos-forecasting yfinance pandas-ta plotly google-genai python-dotenv -q

## 2️⃣ Import & Config

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
from chronos import ChronosPipeline
import warnings
warnings.filterwarnings('ignore')

# ── Ayarlar ──────────────────────────────────────────
BIST_TICKERS   = ['THYAO.IS', 'GARAN.IS', 'AKBNK.IS', 'EREGL.IS', 'BIMAS.IS',
                   'ASELS.IS', 'KCHOL.IS', 'YKBNK.IS', 'TUPRS.IS', 'SISE.IS']
CRYPTO_TICKERS = ['BTC-USD', 'ETH-USD']
ALL_TICKERS    = BIST_TICKERS + CRYPTO_TICKERS

FORECAST_HORIZON = 5   # Kaç gün ileriye tahmin
CONTEXT_LENGTH   = 60  # Geçmişten kaç günü kullan
NUM_SAMPLES      = 100 # Olasılıksal tahmin için sample sayısı

print('✓ Config hazır')

## 3️⃣ Veri Katmanı: OHLCV + Teknik İndikatörler

`yfinance` ile hisse verisi çekip `pandas-ta` ile RSI, MACD ve Bollinger Bands hesaplıyoruz.

**Dikkat:** `.history()` methodu `.download()`'a göre daha stabil — yfinance'ın yeni versiyonlarında MultiIndex sorunu yaşamamak için `Ticker` objesi üzerinden gidiyoruz.

In [ ]:
def fetch_data(ticker: str, period: str = '6mo') -> pd.DataFrame:
    """Hisse verisini çek ve teknik indikatörleri hesapla."""
    df = yf.Ticker(ticker).history(period=period)

    if df.empty:
        raise ValueError(f'Boş veri: {ticker}')

    # MultiIndex edge case
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    df = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    df.dropna(inplace=True)

    # Teknik indikatörler
    df['RSI']  = ta.rsi(df['Close'], length=14)

    macd_df    = ta.macd(df['Close'])
    df['MACD'] = macd_df.iloc[:, 0]

    bb_df        = ta.bbands(df['Close'], length=20)
    bb_cols      = bb_df.columns.tolist()
    df['BB_up']  = bb_df[bb_cols[0]]
    df['BB_mid'] = bb_df[bb_cols[1]]
    df['BB_low'] = bb_df[bb_cols[2]]

    df.dropna(inplace=True)
    return df

# Verileri çek
print('Veri çekiliyor...')
data = {}
for ticker in ALL_TICKERS:
    try:
        data[ticker] = fetch_data(ticker)
        print(f'  ✓ {ticker} ({len(data[ticker])} bar)')
    except Exception as e:
        print(f'  ✗ {ticker}: {e}')

print(f'\n{len(data)} ticker yüklendi.')
print('\nÖrnek veri (THYAO.IS):')
data['THYAO.IS'].tail(3)

## 4️⃣ Analiz Katmanı: Amazon Chronos

Chronos'un güzel yanı: **herhangi bir numeric time series** alıp probabilistic forecast üretiyor.
Biz bunu iki şekilde kullanıyoruz:

1. **Forecast** → 5 günlük median + %80 güven bandı
2. **Anomaly Score** → Son fiyat, beklenen dağılımdan kaç std uzakta?

Anomaly score'u şöyle yorumla:
- `< 1.0` → Normal
- `1.0 - 2.0` → Dikkat
- `> 2.0` → Anormal hareket
- `> 3.0` → Kritik

In [ ]:
# Chronos modelini yükle
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype  = torch.bfloat16 if device == 'cuda' else torch.float32

print(f'Chronos yükleniyor... (device={device})')
pipeline = ChronosPipeline.from_pretrained(
    'amazon/chronos-t5-small',
    device_map=device,
    dtype=dtype,
)
print('✓ Model hazır')

In [ ]:
def run_forecast(df: pd.DataFrame) -> dict:
    """Chronos ile probabilistic forecast ve anomaly score üret."""
    close_vals = df['Close'].values[-CONTEXT_LENGTH:]
    close      = torch.tensor(close_vals, dtype=torch.float32)

    forecast = pipeline.predict(close, FORECAST_HORIZON, num_samples=NUM_SAMPLES)
    samples  = forecast[0].numpy()  # shape: (num_samples, horizon)

    # Quantile'lar → güven bandı
    low    = np.quantile(samples, 0.10, axis=0)
    median = np.quantile(samples, 0.50, axis=0)
    high   = np.quantile(samples, 0.90, axis=0)

    # Anomaly score hesapla
    last_price    = float(df['Close'].iloc[-1])
    band_q        = np.quantile(samples[:, 0], [0.10, 0.90])
    in_band       = bool(band_q[0] <= last_price <= band_q[1])
    pred_mean     = float(samples[:, 0].mean())
    pred_std      = float(samples[:, 0].std())
    anomaly_score = abs(last_price - pred_mean) / (pred_std + 1e-8)

    return {
        'low': low.tolist(),
        'median': median.tolist(),
        'high': high.tolist(),
        'anomaly_score': round(anomaly_score, 2),
        'in_band': in_band,
        'last_price': last_price,
        'forecast_direction': 'UP' if median[-1] > last_price else 'DOWN',
        'expected_change_pct': round((median[-1] - last_price) / last_price * 100, 2),
    }

# Tüm ticker'lar için forecast hesapla
print('Forecast hesaplanıyor...')
forecasts = {}
for ticker, df in data.items():
    try:
        forecasts[ticker] = run_forecast(df)
        r = forecasts[ticker]
        direction_emoji = '📈' if r['forecast_direction'] == 'UP' else '📉'
        anomaly_emoji   = '🚨' if r['anomaly_score'] >= 2 else '✅'
        print(f'  {direction_emoji} {ticker}: {r["forecast_direction"]} '
              f'{r["expected_change_pct"]:+.1f}% | '
              f'{anomaly_emoji} Anomaly: {r["anomaly_score"]:.2f}')
    except Exception as e:
        print(f'  ✗ {ticker}: {e}')

## 5️⃣ Özet Tablo

In [ ]:
rows = []
for ticker, fc in forecasts.items():
    df   = data[ticker]
    last = df.iloc[-1]
    rows.append({
        'Hisse':          ticker,
        'Son Fiyat':      round(fc['last_price'], 2),
        'Yön':            fc['forecast_direction'],
        '5G Beklenti (%)': fc['expected_change_pct'],
        'RSI(14)':        round(last['RSI'], 1) if 'RSI' in df.columns else '-',
        'Anomaly Skoru':  fc['anomaly_score'],
        'Bantta mı':      '✅' if fc['in_band'] else '⚠️',
    })

summary = pd.DataFrame(rows).set_index('Hisse')

def highlight(val):
    if val == 'UP':   return 'color: green; font-weight: bold'
    if val == 'DOWN': return 'color: red; font-weight: bold'
    if isinstance(val, float):
        if val >= 3:  return 'color: red; font-weight: bold'
        if val >= 2:  return 'color: orange'
        if val > 0:   return 'color: green'
        if val < 0:   return 'color: red'
    return ''

summary.style.applymap(highlight)

## 6️⃣ Görselleştirme: Detay Chart

Candlestick + Bollinger Bands + Chronos Forecast Band + RSI + MACD

In [ ]:
def plot_ticker(ticker: str):
    """Seçilen hisse için interaktif Plotly chart çiz."""
    df = data[ticker]
    fc = forecasts[ticker]

    fig = make_subplots(
        rows=3, cols=1, shared_xaxes=True,
        row_heights=[0.6, 0.2, 0.2],
        subplot_titles=[
            f'{ticker} | {fc["forecast_direction"]} {fc["expected_change_pct"]:+.1f}% | Anomaly: {fc["anomaly_score"]}',
            'RSI(14)', 'MACD'
        ],
        vertical_spacing=0.04,
    )

    # Candlestick
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'], high=df['High'],
        low=df['Low'],   close=df['Close'],
        name='OHLC',
        increasing_line_color='#00ff88',
        decreasing_line_color='#ff4444',
    ), row=1, col=1)

    # Bollinger Bands
    if 'BB_up' in df.columns:
        for col_name, color, lname in [
            ('BB_up',  'rgba(255,170,0,0.35)', 'BB Upper'),
            ('BB_mid', '#ffaa00',               'BB Mid'),
            ('BB_low', 'rgba(255,170,0,0.35)', 'BB Lower'),
        ]:
            fig.add_trace(go.Scatter(
                x=df.index, y=df[col_name],
                line=dict(color=color, width=1),
                name=lname, opacity=0.9,
            ), row=1, col=1)

    # Chronos Forecast
    future_dates = pd.bdate_range(start=df.index[-1], periods=FORECAST_HORIZON + 1)[1:]

    fig.add_trace(go.Scatter(
        x=list(future_dates), y=fc['median'],
        line=dict(color='#00cfff', width=2, dash='dash'),
        name='Forecast Median',
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=list(future_dates) + list(future_dates[::-1]),
        y=fc['high'] + fc['low'][::-1],
        fill='toself', fillcolor='rgba(0,207,255,0.08)',
        line=dict(color='rgba(0,0,0,0)'),
        name='%80 Güven Bandı',
    ), row=1, col=1)

    # RSI
    if 'RSI' in df.columns:
        fig.add_trace(go.Scatter(
            x=df.index, y=df['RSI'],
            line=dict(color='#a855f7', width=1.5), name='RSI',
        ), row=2, col=1)
        for y, color, dash in [(70, 'red', 'dash'), (30, 'green', 'dash'), (50, 'gray', 'dot')]:
            fig.add_hline(y=y, line_dash=dash, line_color=color, row=2, col=1)

    # MACD
    if 'MACD' in df.columns:
        fig.add_trace(go.Scatter(
            x=df.index, y=df['MACD'],
            line=dict(color='#facc15', width=1.5), name='MACD',
        ), row=3, col=1)
        fig.add_hline(y=0, line_dash='dash', line_color='gray', row=3, col=1)

    fig.update_layout(
        template='plotly_dark',
        paper_bgcolor='#0a0a0a',
        plot_bgcolor='#0f0f0f',
        height=700,
        showlegend=False,
        xaxis_rangeslider_visible=False,
        margin=dict(l=40, r=20, t=40, b=20),
        font=dict(color='#e0e0e0'),
    )
    fig.show()

# İncelemek istediğin hisseyi seç
plot_ticker('THYAO.IS')

In [ ]:
# Diğer hisseler için de çalıştır
plot_ticker('GARAN.IS')

In [ ]:
plot_ticker('BTC-USD')

## 7️⃣ Sonuçlar & Değerlendirme

### Ne öğrendik?

**Chronos'un güçlü yanları:**
- Zero-shot çalışıyor — fine-tune gerektirmiyor
- Probabilistic output → sadece nokta tahmin değil, dağılım veriyor
- Anomaly detection için güçlü bir baseline

**Dikkat edilmesi gerekenler:**
- Fiyat tahmini için tek başına yeterli değil
- Regime change'i (kriz, büyük haberler) öngöremiyor
- En iyi kullanım: ensemble içinde bir sinyal veya anomaly detector olarak

### Geliştirme Fikirleri
- Alert sistemi: Anomaly score > 2 olduğunda bildirim
- Haber sentiment + Chronos forecast kombinasyonu
- Walk-forward validation ile gerçek performans testi
- Edge deployment için model quantization

---
**Full Streamlit Dashboard** için GitHub reposuna bakın.